# Install and Import Libraries

In [1]:
# %pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
# %pip install langchain-huggingface
# %pip install -qU langchain-qdrant



In [2]:
import torch

torch.cuda.is_available()

True

In [3]:
!nvidia-smi

Mon Jun  9 18:59:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-16GB           On  |   00000000:18:00.0 Off |                    0 |
| N/A   39C    P0             43W /  300W |       4MiB /  16384MiB |      0%   E. Process |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import os
import sys
import argparse
import io
from PIL import Image
from tqdm import tqdm
import json
import pandas as pd
import numpy as np

from datasets import load_dataset

import torch
from torch.utils.data import Dataset, DataLoader
# from colpali_engine.models import ColQwen2, ColQwen2Processor
# from transformers.utils.import_utils import is_flash_attn_2_available

from langchain_text_splitters import RecursiveCharacterTextSplitter

/projectnb/vkolagrp/avarshn/.conda/envs/rag_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Text Splitter

In [5]:


# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,  # chunk size (characters)
#     chunk_overlap=200,  # chunk overlap (characters)
#     add_start_index=True,  # track index in original document
# )
# # all_splits = text_splitter.split_documents(docs)

# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size=100,
#     chunk_overlap=20,
#     length_function=len,
#     is_separator_regex=False,
# )
# # texts = text_splitter.create_documents([state_of_the_union])


# print(texts[0])
# print(texts[1])
# print(f"Split blog post into {len(all_splits)} sub-documents.")

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=20,
    chunk_overlap=10,
    length_function=len,
#     is_separator_regex=False,
#     add_start_index=True
)

In [7]:
text_splitter.create_documents(["123456789123456789gjdfddsasjghj34567 Kitty \n\ncat"])

[Document(metadata={}, page_content='123456789123456789gj'),
 Document(metadata={}, page_content='23456789gjdfddsasjgh'),
 Document(metadata={}, page_content='dfddsasjghj34567'),
 Document(metadata={}, page_content='Kitty'),
 Document(metadata={}, page_content='cat')]

In [48]:
metas = [{"source": "file1.txt"}, {"source": "file2.txt"}]
all_splits = text_splitter.create_documents(["1256 7 Kitty \n\ncat", "Hi H ow are you doing?"], metadatas = metas)
all_splits

[Document(metadata={'source': 'file1.txt', 'start_index': 0}, page_content='1256 7 Kitty \n\ncat'),
 Document(metadata={'source': 'file2.txt', 'start_index': 0}, page_content='Hi H ow are you doing?')]

In [49]:
for split in all_splits:
    if "source" in split.metadata:
        split.metadata.update(split.metadata["source"])
        del split.metadata["source"]

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [9]:

text_splitter.create_documents(["This is a demo sentence. Here is another."])

[Document(metadata={}, page_content='This is a demo'),
 Document(metadata={}, page_content='is a demo sentence.'),
 Document(metadata={}, page_content='sentence. Here is'),
 Document(metadata={}, page_content='Here is another.')]

In [10]:
text_splitter.create_documents(["Meow Moneowhjgjdfdfifgfgfgfgdfdfdfhgasasasasasjghj Kitty cat"])

[Document(metadata={}, page_content='Meow'),
 Document(metadata={}, page_content='Moneowhjgjdfdfifgfg'),
 Document(metadata={}, page_content='jdfdfifgfgfgfgdfdfdf'),
 Document(metadata={}, page_content='fgfgdfdfdfhgasasasas'),
 Document(metadata={}, page_content='hgasasasasasjghj'),
 Document(metadata={}, page_content='Kitty cat')]

# LLM Model - For generation

In [11]:
# import getpass
# import os

# if not os.environ.get("PPLX_API_KEY"):
#   os.environ["PPLX_API_KEY"] = getpass.getpass("Enter API key for Perplexity: ")

# from langchain.chat_models import init_chat_model

# llm = init_chat_model("llama-3.1-sonar-small-128k-online", model_provider="perplexity")

# Embedding Models

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

model_checkpoint = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name = model_checkpoint)

# If there is some way to get from embedding model object itself 
embedding_dim = 768

In [13]:
print(embeddings)

model_name='sentence-transformers/all-mpnet-base-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False


# Vector Store / Vector Database

In [45]:
# from langchain_qdrant import QdrantVectorStore
# from qdrant_client import QdrantClient

# client = QdrantClient(":memory:")
# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="test",
#     embedding=embeddings,
# )

# ValueError: Collection test not found

In [46]:
# from qdrant_client import QdrantClient
# from qdrant_client.models import VectorParams, Distance

# # from langchain.embeddings import FakeEmbeddings  # or your own embedding model




# # Step 2: Create collection manually
# client.recreate_collection(
#     collection_name="test",
#     vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
# )



In [14]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

from langchain_qdrant import QdrantVectorStore

# # Initialize your client
# client = QdrantClient(host="localhost", port=6333)

# Step 1: Create Qdrant client in memory
client = QdrantClient(":memory:")

# Step 2: Define collection name and configuration
collection_name = "test"
vector_config = VectorParams(size=embedding_dim, distance=Distance.COSINE)

# Check if collection exists, create it if not
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=vector_config
    )

# Step 3: Use it with LangChain
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)

# Dataset

In [15]:
# import pyarrow.parquet as pq

In [16]:
# Read a Parquet file
ds = pd.read_parquet("../data_task1/MMDocIR_pages.parquet")

# Show first few rows
print(ds.head())

       doc_name          domain passage_id                      image_path  \
0  2310.05634v2  Academic paper          0  page_images/2310.05634v2_0.jpg   
1  2310.05634v2  Academic paper          1  page_images/2310.05634v2_1.jpg   
2  2310.05634v2  Academic paper          2  page_images/2310.05634v2_2.jpg   
3  2310.05634v2  Academic paper          3  page_images/2310.05634v2_3.jpg   
4  2310.05634v2  Academic paper          4  page_images/2310.05634v2_4.jpg   

                                        image_binary  \
0  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...   
1  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...   
2  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...   
3  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...   
4  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...   

                                            ocr_text  \
0  Towards Verifiable Generation: A Benchmark for...   
1  for attribution? Many real-world applications ...   
2  be partially verified b

In [17]:
# # Open Parquet file
# parquet_file = pq.ParquetFile("../data_task1/MMDocIR_pages.parquet")

# # Check number of row groups
# print("Row Groups:", parquet_file.num_row_groups)

# # Iterate over row groups
# for i in range(parquet_file.num_row_groups):
#     batch = parquet_file.read_row_group(i)
#     df_chunk = batch.to_pandas()
#     print(df_chunk.shape)

In [18]:
class TextDataset(Dataset):
    def __init__(self, dataset, doc_name_filter):
        """
        Args:
            dataset: Parquet dataset as Pandas.DataFrame object
            doc_name_filter: Only process text with this `doc_name`
        """
        filtered_dataset = dataset[dataset["doc_name"] == doc_name_filter]
        self.data = filtered_dataset.reset_index(drop = True)
        self.doc_name = doc_name_filter

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        passage_id = self.data.iloc[idx]["passage_id"]
        text = self.data.iloc[idx]["vlm_text"]     # "ocr_text"
        return passage_id, text

    def __str__(self):
        return f"Contains all the pages of this doc_id : {self.doc_name}"


def text_collate_fn(batch):
    """
    Custom collate function to ensure DataLoader returns:
    - A list of passage_ids (strings)
    - A list of OCR text (either from OCR or VLM-based models)
    """
    passage_ids, texts = zip(*batch)

    [{"page_source": "file1.txt"}, {"source": "file2.txt"}]

    return list(passage_ids), list(texts)

def process_document_texts(ds, doc_name):
    """
    Process texts only from the document that matches `doc_name`.
    """
    text_dataset = TextDataset(ds, doc_name)
    text_dataloader = DataLoader(
        text_dataset, 
        batch_size=4, 
        shuffle=False,
        collate_fn=text_collate_fn
    )

    for passage_ids, texts in tqdm(text_dataloader, desc="Processing document texts"):

        metas = [{"page_source" : passage_id, "doc_source" : doc_name} for passage_id in passage_ids]

        all_splits = text_splitter.create_documents(texts, metadatas = metas)

        document_ids = vector_store.add_documents(documents=all_splits)

NameError: name 'all_splits' is not defined

In [19]:
# def get_relevant_passage_topk(query_text, embedding_store, passage_store, top_k=5):
#     """
#     Retrieves the top-k most relevant passages for a given query.
    
#     Args:
#         query_text (str): Query `question`
#         top_k (int): Number of top passages to retrieve.
    
#     Returns:
#         list: List of passage IDs for the top-k matches.
#     """

#     batch_queries = processor.process_queries([query_text]).to(model.device)
#     with torch.no_grad():
#         query_embeddings = model(**batch_queries)
#     scores = processor.score_multi_vector(query_embeddings, embedding_store)

#     min_k = min(top_k, scores[0].numel())      # To avoid out of range errors if documents has less than top_k pages
#     top_k_scores, top_k_indices = scores[0].topk(min_k)   
#     top_k_passages = [passage_store[idx] for idx in top_k_indices.tolist()]

#     while len(top_k_passages) < top_k:
#         top_k_passages.append(top_k_passages[-1])

#     return top_k_passages

In [20]:
script_testing = False

if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument("--start_idx", type=int, required=True, help="Start index for dataset slicing.")
    parser.add_argument("--end_idx", type=int, required=True, help="End index for dataset slicing.")
    parser.add_argument("--model_checkpoint", type = str, required = True, help = "Hugging Face model checkpoints")
    if not script_testing:
        test_args = ["--start_idx", "0", "--end_idx", "20", "--model_checkpoint", "all-mpnet-base-v2"]
        args = parser.parse_args(test_args)
    else:
        args = parser.parse_args()


    # Load dataset - Read a Parquet file
    ds = pd.read_parquet("../data_task1/MMDocIR_pages.parquet")


    # Use this for querying purpose
    # Process JSONL data and update passage_id field with relevant passages
    data_json = []
    with open("../data_task1/MMDocIR_gt_remove.jsonl", 'r', encoding="utf-8") as f:
        for line in f:
            data_json.append(json.loads(line.strip()))

    data_json = data_json[args.start_idx:args.end_idx]

    print(f"Total Questions: {len(data_json)}")

    data_df = pd.DataFrame(data_json)

    doc_list = list(data_df["doc_name"].unique())

    res_passage_ids = []     # page numbers stating with zero index
    res_question_ids = []
    res_questions = []

    # Text- Splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # chunk size (characters)
        chunk_overlap=200,  # chunk overlap (characters)
        add_start_index=True,  # track index in original document
        length_function=len,
        is_separator_regex=False,
    )

Total Questions: 20


In [21]:
# data_df

In [22]:
text_dataset = TextDataset(ds, doc_list[0])

In [23]:
print(text_dataset)

Contains all the pages of this doc_id : 2310.05634v2


In [24]:
len(text_dataset)

24

In [25]:
text_dataset.data.loc[0]

doc_name                                             2310.05634v2
domain                                             Academic paper
passage_id                                                      0
image_path                         page_images/2310.05634v2_0.jpg
image_binary    b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
ocr_text        Towards Verifiable Generation: A Benchmark for...
vlm_text        Towards Verifiable Generation: A Benchmark for...
Name: 0, dtype: object

# To view the Vector Database

In [42]:
from langchain.schema import Document

points, _ = client.scroll(
    collection_name="test",
    scroll_filter=None,
    limit=200,
    with_vectors=False  # must be True to get vectors
)

documents = [
    Document(page_content=point.payload.get("page_content", ""), metadata=point.payload)
    for point in points
]

print(points[0])

id='000344aa775c480698f942573587e221' payload={'page_content': '5.4 Human Evaluation \nWe conduct human evaluation to verify the correla- tion between automatic evaluation and human judg- ment. We randomly sample 100 sentence-citation pairs from each of the three baselines: ChatGPT (temperature 0.5), LLaMA-7B, and Vicuna-13B. We request two proficient English annotators for each baseline to determine if the citation aligns to the sentence and provides support for it. The reason we choose metric alignment here is in appendix  C , with instruction to annotators and IAA. \nThe comparison between automatically calcu- lated Alignment and human evaluation results is shown in Table  6 . For all three baselines, the auto- matic and human scores are close with a gap within 2.5, despite the significant differences among the baselines. This indicates a strong correlation be- tween the automatically calculated alignment and human judgments. The experiment results demon- strate that the automatic e

In [43]:
len(points)

135

In [50]:
points[0]


Record(id='000344aa775c480698f942573587e221', payload={'page_content': '5.4 Human Evaluation \nWe conduct human evaluation to verify the correla- tion between automatic evaluation and human judg- ment. We randomly sample 100 sentence-citation pairs from each of the three baselines: ChatGPT (temperature 0.5), LLaMA-7B, and Vicuna-13B. We request two proficient English annotators for each baseline to determine if the citation aligns to the sentence and provides support for it. The reason we choose metric alignment here is in appendix  C , with instruction to annotators and IAA. \nThe comparison between automatically calcu- lated Alignment and human evaluation results is shown in Table  6 . For all three baselines, the auto- matic and human scores are close with a gap within 2.5, despite the significant differences among the baselines. This indicates a strong correlation be- tween the automatically calculated alignment and human judgments. The experiment results demon- strate that the aut

# Process the documents

In [51]:
from qdrant_client import models

In [61]:
for doc_name in doc_list:
    print("*-"*20)
    print(f"Doc Name: {doc_name}")

    # Indexing Phase
    process_document_texts(ds, doc_name)

    # Querying Phase
    filtered_df = data_df[data_df["doc_name"] == doc_name]
    
    top_k = 15
    for row_idx, row_data in filtered_df.iterrows():

        top_k_matches = vector_store.similarity_search(
            query = row_data["question"],
            k=top_k,
            # Filter on metadata key "doc_source"
            filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="metadata.doc_source",
                        match=models.MatchValue(value=doc_name)
                    )
                ]
            )
        )

        print(row_data)

        while len(top_k_matches) < top_k:
            top_k_matches.append(top_k_matches[-1])

        res_passage_ids.append(top_k_matches)

        res_question_ids.append(row_data['question_id'])
        res_questions.append(row_data['question'])


# Save the results

res_df = pd.DataFrame({"question_id": res_question_ids, "question" : res_questions, "passage_id": res_passage_ids })

result_dir = f"../results_task1/{model_checkpoint}/"

os.makedirs(result_dir, exist_ok = True)

res_df.to_csv(f"{result_dir}/task1_{args.start_idx}.csv", index = False)
print("Saved dataset with passage_ids")

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Doc Name: 2310.05634v2


Processing document texts: 100%|██████████| 6/6 [00:00<00:00,  7.07it/s]


question_id                                                 10000
question        In figure 1, which relation arrows do not poin...
doc_name                                             2310.05634v2
domain                                             Academic paper
num_of_pages                                                   24
passage_id                                                     []
Name: 0, dtype: object
question_id                                                 10001
question        In figure 5, what is the color of the line tha...
doc_name                                             2310.05634v2
domain                                             Academic paper
num_of_pages                                                   24
passage_id                                                     []
Name: 1, dtype: object
question_id                                         10002
question        How many tables include "F1" as a metric?
doc_name                                     2

Processing document texts: 100%|██████████| 6/6 [00:00<00:00,  7.27it/s]


question_id                                                 10006
question        In figure 4, which nodes are retrieved by RAPT...
doc_name                                             2401.18059v1
domain                                             Academic paper
num_of_pages                                                   23
passage_id                                                     []
Name: 6, dtype: object
question_id                                                 10007
question        What are the horizontal and vertical axis of F...
doc_name                                             2401.18059v1
domain                                             Academic paper
num_of_pages                                                   23
passage_id                                                     []
Name: 7, dtype: object
question_id                                                 10008
question        Based on "PSEUDOCODE FOR RETRIEVAL METHODS", w...
doc_name                      

Processing document texts: 100%|██████████| 6/6 [00:00<00:00,  7.13it/s]


question_id                                                 10012
question        In the figure that has a tree shape, what is t...
doc_name                                             2312.10997v5
domain                                             Academic paper
num_of_pages                                                   21
passage_id                                                     []
Name: 12, dtype: object
question_id                                              10013
question        In figure 3, how many distinct icons are used?
doc_name                                          2312.10997v5
domain                                          Academic paper
num_of_pages                                                21
passage_id                                                  []
Name: 13, dtype: object
question_id                                                 10014
question        What is the paper's full title that proposes t...
doc_name                                      

Processing document texts: 100%|██████████| 5/5 [00:00<00:00, 18.09it/s]

question_id                                10018
question           What degree does LEBOUR have?
doc_name        fd76bbefe469561966e5387aa709c482
domain                            Academic paper
num_of_pages                                  17
passage_id                                    []
Name: 18, dtype: object
question_id                                         10019
question        What is the title of the of the Figure 2?
doc_name                 fd76bbefe469561966e5387aa709c482
domain                                     Academic paper
num_of_pages                                           17
passage_id                                             []
Name: 19, dtype: object
Saved dataset with passage_ids


In [62]:
top_k_matches

[Document(metadata={'page_source': '2', 'doc_source': 'fd76bbefe469561966e5387aa709c482', 'start_index': 703, '_id': '7ae63d8c27dc471f8b08c7ef87a5ffea', '_collection_name': 'test'}, page_content='The image is a diagram displaying "Breccia Gashes," which are formations within a cliff. The top part of these gashes is shown as being eroded or "denuded," while the lower portion is visible within the cliff face. "Breccia" refers to a rock composed of broken fragments of minerals or rock cemented together by a fine-grained matrix, which is depicted in the diagram as a pattern of angular fragments, representing the texture of the breccia within these formations.\nDia g rum of Breccia Gash in Cli fr with top denuded of fund Bottom concealed by the beach. \nThe image is a line drawing labeled "FiG. 3," depicting a geological cross-section with features that resemble vertical geological formations or structures. These structures could represent elements like veins or intrusions within layered ro

In [74]:
ds = pd.read_parquet("../data_task1/MMDocIR_pages.parquet")

In [75]:
ds

,doc_name,domain,passage_id,image_path,image_binary,ocr_text,vlm_text
0,2310.05634v2,Academic paper,0,page_images/2310.05634v2_0.jpg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Towards Verifiable Generation: A Benchmark for...,Towards Verifiable Generation: A Benchmark for...
1,2310.05634v2,Academic paper,1,page_images/2310.05634v2_1.jpg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,for attribution? Many real-world applications ...,"\nTo address the first challenge, we utilize k..."
2,2310.05634v2,Academic paper,2,page_images/2310.05634v2_2.jpg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,be partially verified by the knowledge graph G...,\n2.2 Dataset Construction \nEach entry of dat...
3,2310.05634v2,Academic paper,3,page_images/2310.05634v2_3.jpg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,evance on general and specific settings are 73...,"\nIn practice, it is difficult to define a pre..."
4,2310.05634v2,Academic paper,4,page_images/2310.05634v2_4.jpg,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Model Output: Mininum\nSentence’ [k1][k2]. Kno...,The image shows a diagram explaining how to ev...
...,...,...,...,...,...,...,...
20390,disciplined-agile-business-analysis-1602180127...,Tutorial/Workshop,36,page_images/disciplined-agile-business-analysi...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,My parting advice....\n\nRecognize that Agile ...,My parting advice. \nRecognize that Agile and ...
20391,disciplined-agile-business-analysis-1602180127...,Tutorial/Workshop,37,page_images/disciplined-agile-business-analysi...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Thank You!\n\nscott [at] scottambler.com\nwW @...,Thank You! \nscott[at]scott ambler.com @scott ...
20392,disciplined-agile-business-analysis-1602180127...,Tutorial/Workshop,38,page_images/disciplined-agile-business-analysi...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Shuhari and Disciplined Agile Certification\n\...,Shuhariand Disciplined Agile Certification \nT...
20393,disciplined-agile-business-analysis-1602180127...,Tutorial/Workshop,39,page_images/disciplined-agile-business-analysi...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Would You Like This Presented\nto Your Organiz...,Would You Like This Presented to Your Organiza...
